# Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa. 
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt** 
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**

In [70]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [146]:
class NaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        train_features = self.data_discretization(train_features)
        class_counts = Counter(train_classes)
        num_samples = len(train_classes)
        for c in class_counts:
            self.priors[c] = class_counts[c] / num_samples
        
        for c in self.priors:
            self.likelihoods[c] = {}
            for i in range(train_features.shape[1]):
                feature_values = train_features[train_classes == c, i]
                self.likelihoods[c][i] = dict(Counter(feature_values))
                for value in self.likelihoods[c][i]:
                    self.likelihoods[c][i][value] /= class_counts[c]

    def data_discretization(self, data):
        for i in range(data.shape[1]):
            bin_size = 5
            data_min = min(data[:,i])
            data_max = max(data[:,i])
            bins = np.linspace(data_min, data_max, bin_size)
            data[:,i] = np.digitize(data[:,i], bins, right=True)
        return data

    def predict(self, sample):
        posteriors = {}
        for c in self.priors:
            posteriors[c] = self.priors[c]
            for i in range(sample.shape[0]):
                feature_value = sample[i]
                if feature_value in self.likelihoods[c][i]:
                    posteriors[c] *= self.likelihoods[c][i][feature_value]
                else:
                    posteriors[c] *= 0
        return max(posteriors, key=posteriors.get)
    
    def _test(self, x, y, test_size):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
        self.build_classifier(x_train, y_train)
        predictions = []
        x_test = self.data_discretization(x_test)
        for sample in x_test:
            predictions.append(self.predict(sample))
        return sum(predictions == y_test) / len(y_test)



class GaussianNaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        self.classes = list(set(train_classes))
        for c in self.classes:
            self.priors[c] = Counter(train_classes)[c] / len(train_classes)
            self.likelihoods[c] = []
            for i in range(train_features.shape[1]):
                feature_values = train_features[train_classes==c, i]
                mean = np.mean(feature_values)
                std = np.std(feature_values)
                self.likelihoods[c].append((mean, std))

    def normal_dist(self, x, mean, std):
        return (1 / (std * math.sqrt(2 * math.pi))) * math.exp(-((x - mean) ** 2) / (2 * std ** 2))

    def predict(self, test):
        pred = []
        for sample in test:
            predictions = {}
            for c in self.classes:
                predictions[c] = self.priors[c]
                for i in range(sample.shape[0]):
                    predictions[c] *= self.normal_dist(sample[i], self.likelihoods[c][i][0], self.likelihoods[c][i][1])
            pred.append(max(predictions, key=predictions.get))
        return pred

    def _test(self, x, y, test_size):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
        self.build_classifier(x_train, y_train)
        predictions = self.predict(x_test)
        return sum(predictions == y_test) / len(predictions)


In [145]:
gnb = GaussianNaiveBayes()
print(gnb._test(x, y, 0.1))
nb = NaiveBayes()
print(nb._test(x, y, 0.1))

1.0
0.9333333333333333


In [137]:
nb = NaiveBayes()
gnb = GaussianNaiveBayes()
nb_mean = [0, 0, 0, 0, 0]
gnb_mean = [0, 0, 0, 0, 0]
tests_size = [0.1, 0.2, 0.3, 0.4, 0.5]
samples = 1000
for i in range(samples):
    nb_mean[i%len(tests_size)] += nb._test(x, y, tests_size[i%len(tests_size)])
    gnb_mean[i%len(tests_size)] += gnb._test(x, y, tests_size[i%len(tests_size)])
for i in range(len(tests_size)):
    print("test_size: ", tests_size[i])
    print("Naive -- mean accuracy:", nb_mean[i] *len(tests_size)/samples * 100, "%")
    print("Naive Gauss -- mean accuracy:", gnb_mean[i]*len(tests_size)/samples*100, "%")

test_size:  0.1
Naive -- mean accuracy: 83.63333333333338 %
Naive Gauss -- mean accuracy: 95.46666666666677 %
test_size:  0.2
Naive -- mean accuracy: 86.48333333333343 %
Naive Gauss -- mean accuracy: 95.16666666666683 %
test_size:  0.3
Naive -- mean accuracy: 86.6666666666666 %
Naive Gauss -- mean accuracy: 95.32222222222211 %
test_size:  0.4
Naive -- mean accuracy: 86.8583333333334 %
Naive Gauss -- mean accuracy: 95.18333333333327 %
test_size:  0.5
Naive -- mean accuracy: 87.23333333333333 %
Naive Gauss -- mean accuracy: 95.27333333333335 %


Wnioski:
dla różnych rozmiarów testów Gauss średnio działa zawsze lepiej.
Średnia dla gausa wynosi około 95% skuteczność, zaś dla naiwnego wynosi ok 86%